# Plantilla de desarrollo para primer examen parcial

**Pautas:**
- La presente plantilla es un EJEMPLO de cómo ordenar el código de tu examen
- Tienes la libertad DE AGREGAR todos los métodos y secciones en el examen que consideres necesarias
- Realizar el desarrollo por medio de métodos, por ejemplo, ReadInfo(), TrainModel(), etc 
- Los métodos deberán de estar lo mas claro y modularizados que sea posible
- Realizar la documentación de cada método por medio de comentarios y DocStrings 
- Deberás de utilizar un modelo de ML o algún ensamble de os mismos (SVC, DT, NB, KNN, etc)
- Recuerda que puedes usar un split de los datos para entrenamiento y validación
- Puedes revisar la documentación de Sklearn, o la librría que decidas utilizar para entender los parámetros de entrenamiento de los modelos
- NO está permitido el uso de modelos de Deep Learning (DNN, CNN, LSTM, etc.) NI el uso de embeddings

## Librerías a utilizar

In [21]:
# Librerías profe
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer

# Librerías actuales
import pandas as pd
import random
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/eubgo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eubgo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eubgo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Lectura de Dataset

In [22]:
# Aquí cargamos la información del DataSet de entrenamiento
def read_corpus(path):
    """Este método lee los de datos del corpus y los pasa a un dataFrame

    Args:
        path (string): Ubicación del archivo de entrada (Corpus)
    """
    df = pd.read_csv(path)
    df.index = np.arange(1, len(df) + 1)
    print("Elementos en el DataSet:", len(df))
    return df

## Feature Engineering

In [23]:
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2))
# Longitud: numero de palabras en texto
# Diversidad léxica: palabras únicas vs palabras totales en el texto
def lexical_diversity(text):
    return len(set(text)) / len(text) if len(text) > 0 else 0.0

## Data Augmentation

In [24]:

def get_synonyms(word):
    """Obtiene sinónimos de una palabra usando WordNet.

    Args:
        word (str): Palabra para la cual obtener sinónimos.

    Returns:
        list: Lista de sinónimos.
    """
    from nltk.corpus import wordnet
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def replace_with_synonym(sentence):
    """Reemplaza palabras en una oración con un sinónimo aleatorio.

    Args:
        sentence (str): Oración a modificar.

    Returns:
        str: Oración modificada.
    """
    words = sentence.split()
    for i, word in enumerate(words):
        synonyms = get_synonyms(word)
        if synonyms:
            words[i] = random.choice(synonyms).replace("_", " ")
    return ' '.join(words)

def augment(df):
    df_augmented = df.copy()
    clickbait_rows = df_augmented[df_augmented["label"] == "clickbait"]
    clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)
    df_combined = pd.concat([df, clickbait_rows], ignore_index=True)
    return df_combined


## Data reduction

In [25]:
def dataReduction(df):
    """Método para reducir el número de datos en el dataFrame e igualar news y clickbait

    Args:
        df (dataframe): Dataframe a reducir

    Returns:
        df: Dataframe reducido
    """
    # Recalcular las variables news y clickbait después de leer el nuevo DataFrame
    news = df[df["label"] == "news"]
    clickbait = df[df["label"] == "clickbait"]

    # Reducir news para igualar el número de news y clickbait
    news = news.sample(len(clickbait))
    df = pd.concat([news, clickbait], ignore_index=True)

    # # Contar número de news y clickbait
    # news = df[df["label"] == "news"]
    # clickbait = df[df["label"] == "clickbait"]
    # print("news:", len(news))
    # print("clickbait:", len(clickbait))

    return df

## Preprocesamiento

In [26]:
def preprocess(df):
    """Método para preprocesar el texto y agregar características adicionales

    Args:
        df (dataframe): Dataframe a aplicar transformaciones

    Returns:
        dataframe: Dataframe transformado con características adicionales
    """
    
    for column in df.columns:
        df[column] = df[column].str.lower()

    stop_words = stopwords.words("english")
    df["title"] = df["title"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))

    df["title"] = df["title"].str.replace("[^\w\s]", "")

    df["title"] = df["title"].apply(nltk.word_tokenize)

    lemmatizer = WordNetLemmatizer()

    df["title"] = df["title"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x if not word.isdigit()])
    
    return df


## Entrenamiento del modelo

In [27]:
# def train_model(x, y):
#     """Este método realiza el entrenamiento del modelo (Ejemplo)

#     Args:
#         x (list): Lista con los textos a transformar
#         y (list): Lista con los valores de y (Salida)

#     Returns:
#         model: Modelo entrenado
#     """
#     # Parámetros para GridSearchCV
#     param_grid = {
#         'n_estimators': [100, 200, 300],
#         'max_depth': [None, 10, 20, 30],
#         'min_samples_split': [2, 5, 10]
#     }

#     grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
#     grid_search.fit(x, y)

#     # Entrenando el modelo con los mejores parámetros
#     best_model = grid_search.best_estimator_
#     best_model.fit(x, y)
#     return best_model

def train_model(x, y):
    """Este método realiza el entrenamiento del modelo

    Args:
        x (array): Matriz de características (TF-IDF)
        y (list): Lista con los valores de y (Salida)

    Returns:
        model: Modelo entrenado de Regresión Logística
    """
    model = LogisticRegression()
    model.fit(x, y)
    return model


## Validación del modelo

In [28]:
def validate_model(y_test, predicciones):
    # Impresión de matriz de confusión
    # print("Matriz de confusión:")
    # print(confusion_matrix(Y_test, Predicciones))

    # Impresión de procentaje de Accuracy del modelo
    print("\nAccuracy del modelo: ")
    print(metrics.accuracy_score(y_test, predicciones))

    # Impresión de las métricas para el modelo
    print("\nMétricas de evaluación:")
    print(classification_report(y_test, predicciones))

## Pipeline de todo el proceso

In [29]:
path = 'DataSet para entrenamiento del modelo.csv'

# Carga la información y crea un DataFrame
df = read_corpus(path)

# Data augmentation (si lo deseas)
for i in range(2):
    df = augment(df)

# Preprocesamiento
df_pre = preprocess(df)

# División de los datos
X_text = df_pre['title'].values.tolist()
X_text = [' '.join(doc) for doc in X_text]
y = df_pre['label'].values.tolist()

# Crea el vectorizador TF-IDF
tfidf_vect = TfidfVectorizer(ngram_range=(1, 2))
X_tfidf = tfidf_vect.fit_transform(X_text)

# Añade las características adicionales (longitud del texto y diversidad léxica)
df_pre['text_length'] = df_pre['title'].apply(len)
df_pre['lexical_diversity'] = df_pre['title'].apply(lexical_diversity)

# Divide los datos en conjuntos de entrenamiento y prueba
X_train_tfidf, X_test_tfidf, X_train_feats, X_test_feats, y_train, y_test = train_test_split(
    X_tfidf, df_pre[['text_length', 'lexical_diversity']].values, y,  train_size=0.1, test_size=0.02, random_state=42)

# Combina las características TF-IDF y las características adicionales
X_train_combined = np.hstack((X_train_tfidf.toarray(), X_train_feats))
X_test_combined = np.hstack((X_test_tfidf.toarray(), X_test_feats))

# Entrena el modelo (en este caso, Regresión Logística)
model = train_model(X_train_combined, y_train)

# Realiza predicciones
y_pred = model.predict(X_test_combined)

# Calculate and display accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Evalúa el modelo
print(classification_report(y_test, y_pred))

Elementos en el DataSet: 16823


/tmp/ipykernel_2810700/3128748921.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)
/tmp/ipykernel_2810700/3128748921.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clickbait_rows["title"] = clickbait_rows["title"].apply(replace_with_synonym)


## Guardado de modelo

In [ ]:
# Pickle para guardar modelos
import pickle

filename = "model_KikeMau.pickle"

# Guardar el modelo
pickle.dump(model, open(filename, "wb"))


## Prueba del modelo (Parte mas importante)

In [ ]:
# def pipeline(input_file, model):
#     # Cargamos la información y creamos un DataFrame
#     df = read_corpus(path)

#     # Preprocesamiento
#     df_pre = preprocess(df)

#     # Lectura y split de los datos
#     X = df_pre['title'].values.tolist()
#     X = [' '.join(doc) for doc in X]
#     y = df_pre['label'].values.tolist()

#     X = tfidf_vect.fit_transform(X)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.05, test_size=0.01, random_state=42)

#     X_train_tfidf = tfidf_vect.transform(X_train)
#     X_test_tfidf = tfidf_vect.transform(X_test)
    
#     trained_model = train_model(X_train_tfidf, y_train, model)

#     # Impresión de métricas
#     Predicciones = model.predict(X_test)
#     validate_model(y_test, Predicciones)

# model = RandomForestClassifier()

# # Prueba para calificación del examen
# input_file = 'DataSet para entrenamiento del modelo.csv'
# pipeline(input_file, model)